# Update spotify net/assign tracks

> Update spotify net by assigning tracks with above a playcount threshold to an existing public playlist and removing from net

In [1]:
#| default_exp prep_model

In [2]:
# export
import pandas as pd
import requests
import boto3
import json
from io import BytesIO
import joblib
import pickle
import numpy as np
pd.set_option('display.max_columns', None)

In [3]:
#hide
from nbdev.showdoc import *

In [16]:
class ModelPrep:
    def __init__(self):
        self.scaler = None
        self.svd = None
        self.recent_spotify_tracks = None
        self.recent_lastFM_tracks = None
        self.genre_series = None
        self.key_series = None
        
    def load_s3(self):
        '''
        Ok, so it looks like the order in which I'm doing things is:
        1. Load a pretrained scaler
        2. Load a pretrained SVD...model?
        3. Load Spotify data
        4. Load LastFM data
        5. Load a pandas series of genres?
        6. Load a pandas series of eleven keys
        7. Load a pandas series of four time signatures

        Could I instead load a sci-kit learn pipeline?
        '''

        s3_resource = boto3.resource('s3')

        scaler = s3_resource.Object('spotify-net', 'scaler').get()
        scaler = pickle.loads(scaler['Body'].read())

        svd = s3_resource.Object('spotify-net', 'svd').get()
        svd = pickle.loads(svd['Body'].read())

        recent_spotify_tracks = pd.read_csv('s3://spotify-net/newer_tracks.csv', index_col=0)
        recent_lastFM_tracks = pd.read_csv('s3://spotify-net/last_fm_recent_tracks.csv', index_col=0)

        genre_series = pd.read_csv('s3://spotify-net/genres_svd.csv', index_col=0, squeeze=True)

        # These don't need to be loaded from S3, but I'm doing it anyway because...?
        key_series = pd.read_csv('s3://spotify-net/key_list.csv', index_col=0, squeeze=True)
        time_series = pd.read_csv('s3://spotify-net/timeSig_list.csv', index_col=0, squeeze=True)

        self.scaler = scaler
        self.svd = svd
        self.recent_spotify_tracks = recent_spotify_tracks
        self.recent_lastFM_tracks = recent_lastFM_tracks
        self.genre_series = genre_series
        self.key_series = key_series
        self.time_series = time_series

    def merge_frame(self):
        self.recent_spotify_tracks[['name', 'artist']] = self.recent_spotify_tracks[['name', 'artist']].applymap(lambda x: x.upper())
        self.recent_lastFM_tracks[['name', 'artist']] = self.recent_lastFM_tracks[['name', 'artist']].applymap(lambda x: x.upper())

        prepped_frame = pd.merge(self.recent_spotify_tracks, self.recent_lastFM_tracks, on=['name', 'artist'])
        prepped_frame = prepped_frame.drop([
            'playcount', 
            'added at', 
            'artist id',
            'id',
            'type',
            'track_href',
            'analysis_url',
            'type',
            'diff'
        ], axis=1)

        return prepped_frame

In [17]:
m = ModelPrep()
m.load_s3()
p = m.merge_frame()
p


,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id.1,uri,duration_ms,time_signature
0,FINE DAY ANTHEM,SKRILLEX,0.745,0.850,6,-3.903,0,0.0750,0.000285,0.0392,0.1180,0.540,138.030,6tifCCTIVBLC2TmTquYG7G,spotify:track:6tifCCTIVBLC2TmTquYG7G,188261,4
1,ZEBRA,JACKBOY,0.781,0.777,11,-8.193,0,0.3920,0.144000,0.0000,0.0966,0.645,157.001,4sRALTDj1vEOe5dPnPg8Qt,spotify:track:4sRALTDj1vEOe5dPnPg8Qt,110064,4
2,SCHWEIGEN,THE TRUE SPACEMEN,0.715,0.489,4,-7.405,1,0.0383,0.030300,0.4600,0.1180,0.488,130.993,0Db7JyQoWaIiBsKtSHKi54,spotify:track:0Db7JyQoWaIiBsKtSHKi54,204275,4


In [ ]:
# export
def dummies_and_scale(df_classify, constant, scaler):
   # log-transform
   # c = 0.0000001
    c=constant
    df_classify[['speechiness', 'acousticness', 'instrumentalness']] = df_classify[['speechiness', 'acousticness', 'instrumentalness']] + c
    df_classify[['speechiness', 'acousticness', 'instrumentalness']] = np.log(df_classify[['speechiness', 'acousticness', 'instrumentalness']])

   # one-hot
    df_track = pd.get_dummies(df_classify , prefix=['key', 'time_signature'], columns=['key', 'time_signature'])

   # standardScaler
    scale_col = ['danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms']
    df_track[scale_col].head()
    df_track[scale_col] = scaler.transform(df_track[scale_col])

    return df_track

In [ ]:
# export
def full_frame(df_track, gen_series, svd, key_series, time_series):

    curr_gen = df_track.loc[:, df_track.columns.str.startswith('genre_')]
    full_gen = pd.DataFrame(np.zeros((len(df_track), len(gen_series.tolist()))) , columns=gen_series.tolist())
    full_gen = full_gen.add_prefix('genre_')
    full_gen.update(curr_gen)
    full_gen.columns = full_gen.columns.str.replace('genre_', '')

    test_trans = svd.transform(full_gen)
    test_trans = pd.DataFrame(test_trans)
    test_trans = test_trans.add_prefix('genre_')

    df_track = df_track.loc[:, ~df_track.columns.str.startswith('genre_')]
    df_track = pd.concat([df_track, test_trans], axis=1)

    curr_key = df_track.loc[:, df_track.columns.str.startswith('key_')]
    full_key = pd.DataFrame(np.zeros((len(df_track), len(key_series.tolist()))) , columns=key_series.tolist())
    full_key.update(curr_key)

    df_track = df_track.loc[:, ~df_track.columns.str.startswith('key_')]
    df_track = pd.concat([df_track, full_key], axis=1)

    curr_time = df_track.loc[:, df_track.columns.str.startswith('time_signature_')]
    full_time = pd.DataFrame(np.zeros((len(df_track), len(time_series.tolist()))) , columns=time_series.tolist())
    full_time.update(curr_time)

    df_track = df_track.loc[:, ~df_track.columns.str.startswith('time_signature_')]
    df_track = pd.concat([df_track, full_time], axis=1)
    df_track.to_csv('s3://spotify-net/for_prediction.csv')
    print(df_track.shape)

    print('Uploaded to S3')


In [ ]:
# export
if __name__ == '__main__':
    s3_objects = load_s3()
    merged_df = merge_frame(s3_objects['spot_tracks'], s3_objects['last_tracks'])
    transformed = dummies_and_scale(merged_df, 0.0000001, s3_objects['scaler'])
    full_frame(transformed, s3_objects['gen_series'], s3_objects['svd'], s3_objects['key_series'], s3_objects['time_series'])